In [54]:
from functools import partial
from natsort import natsorted
import numpy as np
import os
import pywt
from scipy.signal import welch

#Machine Learning
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

## Feature Engineering

try individual features and see which one fails

In [55]:
def feature_engineering(data_sets, target_dir):
    """
    Function that does the feature engineering and saves the data into according folders.

    Input:
    - data_sets (list): datasets e.g. val_multi & train_multi
    - target_dir: directory where data will be saved
    """
    labels = None

    for data_set in data_sets:
        #load subject_data one by one
        if not os.path.exists(target_dir+"/var"): #check if already preprocessed, else create directories
            os.mkdir(target_dir+"/var")
            os.mkdir(target_dir+"/psd")
            os.mkdir(target_dir+"/dwt")
            os.mkdir(target_dir+"/var_psd_dwt")

        for path in natsorted(os.listdir(data_set))[:-1]:
            file = np.load(os.path.join(data_set,path), allow_pickle=True)

            #VAR
            np_var = partial(np.var, axis=1)
            a, b, c = np.split(file, [55, 110], axis=1)
            VAR = np.hstack((
                np_var(file),
                np_var(a),
                np_var(b),
                np_var(c))
                )
            np.save("{}/{}".format(target_dir+"/var", os.path.basename(path)), VAR)
            #VAR = (VAR-np.mean(VAR))/np.std(VAR) #normalize

            #PSD (Welch)
            freq, power = welch(file, fs = 250)
            PSD = power[:,:25].flatten() #take first 25 components (~35Hz)
            np.save("{}/{}".format(target_dir+"/psd", os.path.basename(path)), PSD)
            #PSD = (PSD-np.mean(PSD))/np.std(PSD) #normalize

            #DWT (only use approximate coefficients)
            (aC, dC1, dC2, dC3) = pywt.wavedec(file, wavelet = "db8", mode='sym', level=3)
            DWT = aC.flatten()
            np.save("{}/{}".format(target_dir+"/dwt", os.path.basename(path)), DWT)
            #DWT = (DWT-np.mean(DWT))/np.std(DWT) #normalize

            sample = np.hstack((VAR, PSD, DWT))
            np.save("{}/{}".format(target_dir+"/var_psd_dwt", os.path.basename(path)), sample)
        
        if labels is not None: #if labels have been loaded for first dataset, hstack and save for each method
            labels = np.hstack((labels, np.load(data_set+"/labels.npy")))
            np.save("{}/labels.npy".format(target_dir), labels)
        else: #for first dataset, labels are loaded into labels variable
            labels = np.load(data_set+"/labels.npy")


## Create Datasets

In [56]:
#for audio-visual
# train_multi = "C:/Users/Daydreamore/Desktop/Semester/BCI/train_multi"
# val_multi = "C:/Users/Daydreamore/Desktop/Semester/BCI/val_multi"
# data_ml_multi = "C:/Users/Daydreamore/Desktop/Semester/BCI/data_ml_multi"
# feature_engineering([val_multi, train_multi], data_ml_multi)

#for visual-only
train_visual = "C:/Users/Daydreamore/Desktop/Semester/BCI/train_visual"
val_visual = "C:/Users/Daydreamore/Desktop/Semester/BCI/val_visual"
data_ml_visual = "C:/Users/Daydreamore/Desktop/Semester/BCI/data_ml_visual"
feature_engineering([val_visual, train_visual], data_ml_visual)

c:\Users\Daydreamore\anaconda3\envs\svenpy\lib\site-packages\scipy\signal\_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 165, using nperseg = 165
  warnings.warn('nperseg = {0:d} is greater than input length '


## Load datasets. 
Randomly select equal number of control trials for each participant to avoid unbalanced dataset.

In [63]:
def load_data(data_path, method, classes):
    """
    Takes care of loading data, undersampling, and label concatenation

    Input:
    - data_path: path from which to load data (e.g. data_ml_multi)
    - method: "var", "psd", "dwt", or "var_psd_dwt"
    - classes (list): classes to load (e.g. [0,1] or [0,2])
    """
    X_train = np.array([np.load("{}/{}/{}".format(data_path, method, path)) for path in natsorted(os.listdir(data_path+"/"+method))[:]])
    Y_train = np.load("{}/labels.npy".format(data_path))
    n_samp = np.where(Y_train == classes[1])[0].shape[0]

    #For control randomly sample n_samp trials (to avoid imbalanced dataset)
    for i in classes:
        target_ixs = np.where(Y_train == i)

        if i == 0: 
            target_ixs_shuffled = np.random.choice(target_ixs[0], size = n_samp, replace = False)
            selected_samples = X_train[target_ixs_shuffled]
            selected_labels = Y_train[target_ixs_shuffled]
            X_train_selected = selected_samples
            Y_train_selected = selected_labels
        else:
            selected_samples = X_train[target_ixs]
            selected_labels = np.ones_like(Y_train[target_ixs])
            X_train_selected = np.concatenate((X_train_selected, selected_samples))
            Y_train_selected = np.concatenate((Y_train_selected, selected_labels))
    return X_train_selected, Y_train_selected

In [84]:
#Vary the following to load the data for each method (var, psd, dwt, or combination) and for each class combination
data_path = "C:/Users/Daydreamore/Desktop/Semester/BCI/data_ml_multi"
method = "var_psd_dwt"
classes = [0,1]
X_train, Y_train = load_data(data_path, method, classes)

Sanity Check to see if dimensions are as expected:

In [85]:
X_train.shape, Y_train.shape

((650, 682), (650,))

## Model Fitting
Try to overfit on training data to see if the SVM can learn to tell the conditions apart.

In [5]:
from sklearn.svm import SVC
overfit = SVC(kernel = "linear")
overfit.fit(X_train, Y_train)
yhat = overfit.predict(X_train)
acc_svm = accuracy_score(Y_train, yhat)
print(acc_svm)

0.8815384615384615


Combine hyperparameter search with model selection through nested cross-validation and put out winning combinations.

In [86]:
outer_results = list()
specificity_results = list()
sensitivity_results = list()
#Outer CV
cv_outer = KFold(n_splits=5, shuffle=True, random_state=42)
for train_ix, val_ix in cv_outer.split(X_train): 
    #split data
    x_train, x_val = X_train[train_ix, :], X_train[val_ix, :]
    y_train, y_val = Y_train[train_ix], Y_train[val_ix]

    #Inner CV
    cv_inner = KFold(n_splits=3, shuffle=True, random_state=42)
    model = SVC()
    #define search space
    param_grid = dict()
    param_grid["C"] = np.logspace(-18, 9, num=9, base=2).tolist() 
    param_grid["kernel"] = ["linear", "sigmoid", "rbf"] 
    param_grid["gamma"] = np.logspace(-18, 9, num=9, base=2).tolist() 
    search = GridSearchCV(model, param_grid, scoring="accuracy", cv=cv_inner, refit=True, error_score="raise")
    result = search.fit(x_train, y_train) #runs fit with all sets of parameters
    best_model = result.best_estimator_ #get the best performing model fit on the whole training set
    yhat = best_model.predict(x_val) #evaluate model on the hold out dataset
    acc_svm = accuracy_score(y_val, yhat) #evaluate the model
    outer_results.append(acc_svm) #store result
    print(">Accuracy=%.3f, est=%.3f, cfg=%s" % (acc_svm, result.best_score_, result.best_params_)) #overall acc and best params

    #sensitivity & specificity
    class_acc = np.bincount(y_val[y_val == yhat])
    class_count = np.bincount(y_val)

    for ix, tclass in enumerate(class_acc):
        if ix == 0:
            condition = "specificity"
        else:
            condition = "sensitivity"

        if tclass == 0:
            if ix == 0:
                specificity = 0.0
                specificity_results.append(specificity)
            else:
                sensitivity = 0.0
                sensitivity_results.append(sensitivity)
            print(f">{condition}: {0.0}")
        else:
            if ix == 0:
                specificity = np.around(tclass/class_count[ix], 3)
                specificity_results.append(specificity)
                print(f">{condition}: {specificity}")
            else:
                sensitivity = np.around(tclass/class_count[ix], 3)
                sensitivity_results.append(sensitivity)
                print(f">{condition}: {sensitivity}")

            
#summarize estimated performance of the model
outer_results = np.array(outer_results)
specificity_results = np.array(specificity_results)
sensitivity_results = np.array(sensitivity_results)
print("----------Final Result----------")
print("Accuracy: %.3f (%.3f)" % (np.mean(outer_results), np.std(outer_results)))
print("Specificity: %.3f (%.3f)" % (np.mean(specificity_results), np.std(specificity_results)))
print("Sensitivity: %.3f (%.3f)" % (np.mean(sensitivity_results), np.std(sensitivity_results)))

>Accuracy=0.915, est=0.931, cfg={'C': 3.9576402424652394e-05, 'gamma': 3.814697265625e-06, 'kernel': 'linear'}
>specificity: 0.943
>sensitivity: 0.883
>Accuracy=0.923, est=0.929, cfg={'C': 0.0004105939527606028, 'gamma': 3.814697265625e-06, 'kernel': 'linear'}
>specificity: 0.917
>sensitivity: 0.931
>Accuracy=0.908, est=0.917, cfg={'C': 3.9576402424652394e-05, 'gamma': 3.814697265625e-06, 'kernel': 'linear'}
>specificity: 0.967
>sensitivity: 0.855
>Accuracy=0.877, est=0.925, cfg={'C': 49.350746413054104, 'gamma': 3.814697265625e-06, 'kernel': 'rbf'}
>specificity: 0.862
>sensitivity: 0.889
>Accuracy=0.954, est=0.929, cfg={'C': 0.004259795830723663, 'gamma': 3.814697265625e-06, 'kernel': 'linear'}
>specificity: 0.969
>sensitivity: 0.939
----------Final Result----------
Accuracy: 0.915 (0.025)
Specificity: 0.932 (0.040)
Sensitivity: 0.899 (0.031)
